# slip velocity in geostrophic flow
Larrry proposed to calculate the slip velocities in a geostrophic background flow as better estimate. Here we plot the solution

In [ ]:
#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

import numpy as np
import xarray as xr 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib import colormaps
# import cartopy.crs as ccrs #for plotting on map
# import cartopy as cart
from datetime import datetime, timedelta


# plotstyle: 
plt.style.use('../python_style_Meike.mplstyle')
import sys
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
from particle_characteristics_functions import buoyancy_drifter, stokes_relaxation_time
from particle_characteristics_functions import Re_particle, diffusion_time
from particle_characteristics_functions import factor_drag_white1991, factor_drag_Schiller1933, factor_drag_morrison2013
from particle_characteristics_functions import drag_length


In [ ]:
d = 0.25 # m diameter outer ring
d_in = 0.2 # m (estimated) dimater inner ring, needed to caluclate volume
h = 0.041  # m (heigth drifter)
m = 0.905 # kg (mass drifter)
cd=0.45 # drag coefficient smooth sphere 
omega_earth =  7.2921e-5 #[rad/sec]
rho_water = 1027 # kg/m3 https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
dynamic_viscosity_water = 1.41 * 10**(-3) # kg/(ms) https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
kinematic_viscosity_water = dynamic_viscosity_water / rho_water
B = buoyancy_drifter(diameter  = d_in, heigth = h, mass = m, density_fluid = rho_water)
tau = stokes_relaxation_time(d, kinematic_viscosity_water, B)
tau_diff = diffusion_time(d_in, kinematic_viscosity_water)

In [ ]:
def solution_uslip_stst(f,R,epsilon,U):
    norm = (1 + f**2 * epsilon**2)
    uslip = (3 * R / 2 - 1) * f**2 * epsilon**2 * U / norm
    vslip = (3 * R / 2 - 1) *f * epsilon *   U / norm
    return uslip, vslip 

def solution_uslip_stst_dim(rho_f, rho_p, d, f, nu, U):
    a = d/2
    R = 2 * rho_f / (rho_f+ 2 *rho_p)

    epsilon =   2 / 9 * (d/2 )**2 / (kinematic_viscosity_water)  * 1/R
    norm =((1/epsilon)**2 + f**2) 
    uslip = 2* (rho_f-rho_p) * f**2 * U / (   (rho_f + 2* rho_p) *  norm ) 
    vslip = 18 * (rho_f-rho_p) * rho_f * nu *  f * U / (  (rho_f+2*rho_p)**2 * a**2 *  norm ) 
    return uslip, vslip 


In [ ]:
def Rep_geostrophic_uslip_stst_dim(B, d, f, nu, U,Rep):
    a = d/2
    R = 2 / (1+ 2 *B)
    crep = factor_drag_white1991(Rep)
    epsilon =   2 / 9 * (d/2 )**2 / (kinematic_viscosity_water * crep)  * 1/R
    norm =((1/epsilon)**2 + f**2) 
    uslip = 2* (1-B) * f**2 * U / (   (1 + 2* B) *  norm ) 
    vslip = 18 * (1-B) * 1 * nu *  f * U / (  (1+2*B)**2 * a**2 *  norm ) 
    Uslip = np.sqrt(uslip**2+vslip**2)
    return Uslip*d/nu

In [ ]:
L = 10000 # m
U = 1 # m/s
R = 2 * rho_water/ (rho_water*( 1+ 2 *B))

epsilon =   2 / 9 * (d/2 )**2 * U / L / (kinematic_viscosity_water)  * 1/R
# print(epsilon)
fdim = 2 * omega_earth * np.sin(54/180 * np.pi)
fdimless =fdim *   L / U
uslip, vslip = solution_uslip_stst(fdimless,R,epsilon,U)
print(uslip)
print(vslip)

In [ ]:

dlist = np.arange(0.001,0.5,0.001)
uslip1, vslip1 = solution_uslip_stst_dim(rho_water, B*rho_water, dlist, fdim, kinematic_viscosity_water, U)


In [ ]:
fig, ax = plt.subplots()
ax.plot(dlist,uslip1,'-',color='firebrick')
ax.plot(dlist,vslip1,'--',color='navy')
ax.legend(['$u_{\\mathrm{slip}}$','$v_{\\mathrm{slip}}$'])
ax.set_xlabel('diameter [m]')
ax.set_ylabel('velocity [U]')

In [ ]:
fig, ax = plt.subplots()
ax.plot(dlist,Re_particle(np.sqrt(uslip1**2+vslip1**2),dlist,kinematic_viscosity_water),'-',color='black')
# ax.plot(dlist,Re_particle(vslip1,d,kinematic_viscosity_water),'--',color='navy')

ax.set_xlabel('diameter [m]')
# ax.set_yscale('log')
# ax.set_xscale('log')
ax.axvline(0.25)
uslip025, vslip025 = solution_uslip_stst_dim(rho_water, B*rho_water, 0.25, fdim, kinematic_viscosity_water, U)
Rep025 = Re_particle(np.sqrt(uslip025**2+vslip025**2),0.25,kinematic_viscosity_water)
print(Rep025)
ax.axhline(Rep025)

# this should be solved selfconsistently (so epsilon should have c(rep) factor)


In [ ]:
fsolve(Rep_geostrophic_uslip_stst_dim(B,0.25,fdim,kinematic_viscosity_water,1,230))

In [ ]:
from scipy.optimize import fsolve
test = []
for d in dlist:
    def fie(Rep):
        return Rep_geostrophic_uslip_stst_dim(B,d,fdim,kinematic_viscosity_water,10,Rep)-Rep
    test.append(fsolve(fie,300)[0])

In [ ]:
fig, ax = plt.subplots()
ax.plot(dlist,test,'-',color='black')
ax.set_xlabel('diameter [m]')
ax.set_ylabel('Re$_p$')
